In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as tick
from IPython.display import HTML, display, Markdown
import tabulate
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import graphviz
from GraphVizUtil import *
from GraphVizFormats import *
import warnings
warnings.filterwarnings('ignore')
import seaborn as sb
sb.set(style='darkgrid', font_scale=1.2)

def filter_descriptions(tech_comm_desc):
    try:
        tech_comm_desc = tech_comm_desc.values[0][0].replace('#', '').replace('"','').replace("\n",'').strip()
    except:
        tech_comm_desc = 'No description provided'
    return tech_comm_desc

def create_args_flowd(df_graph):
    nodes, tech, ltech, to_tech, from_tech = set(), set(), set(), set(), set()
    for ind,row in df_graph.iterrows():
        #descriptions:
        input_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['input_comm'] + "'", con))
        output_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['output_comm'] + "'", con))
        tech_des = filter_descriptions(pd.read_sql("SELECT tech_desc FROM technologies WHERE tech='" + row['tech'] + "'", con))

        if 'ethos' in row['input_comm']:
            ltech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')
        else :
            nodes.add('"' + row['input_comm'] + '"' +  ' [tooltip = "' + input_comm_des + '"]')
        nodes.add('"' + row['output_comm'] + '"' +  ' [tooltip = "' + output_comm_des + '"]')
        tech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')

        if row['input_comm'] != 'ethos':
            to_tech.add('"%s"' % row['input_comm'] + '\t->\t"%s"' % row['tech']) 
        from_tech.add('"%s"' % row['tech'] + '\t->\t"%s"' % row['output_comm'])
    args = dict(
    enodes = "".join('%s;\n\t\t' % x for x in nodes),
    tnodes = "".join('%s;\n\t\t' % x for x in tech),
    iedges = "".join('%s;\n\t\t' % x for x in to_tech),
    oedges = "".join('%s;\n\t\t' % x for x in from_tech),
    snodes = ";".join('%s' %x for x in ltech),
    )
    return args

def return_format_colors():
    colors = {}
    colors.update(getColorConfig(False))
    return colors, quick_run_dot_fmt

def return_flowd_table(final_dem, level=1):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    iterval=0
    if level!=0:
        while len(inputs)>0:
            df_append = df[df['output_comm'].isin(inputs)]
            df_sel = pd.concat([df_sel, df_append])
            inputs = df_append['input_comm'].unique()
            iterval+=1
            if iterval>level-1:
                break
    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

def return_flowd_table_fwds(final_dem):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    outputs = df_sel['output_comm'].unique()

    iterval=0
    while len(inputs)>0:
        df_append = df[df['output_comm'].isin(inputs)]
        df_sel = pd.concat([df_sel, df_append])
        inputs = df_append['input_comm'].unique()
        iterval+=1
        if iterval>2:
            break
    iterval=0
    while len(outputs)>0:
        df_append = df[df['input_comm'].isin(outputs)]
        df_sel = pd.concat([df_sel, df_append])
        outputs = df_append['output_comm'].unique()
        iterval+=1
        if iterval>=0:
            break

    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

con = sqlite3.connect(r'../US_9R_8D.sqlite') #change path to database
cur = con.cursor()   
con.text_factory = str 

def controls_rows(w):
    controls = HBox(w.children[:-1], layout = Layout(flex_flow='row wrap', width='max-content'))
    output = w.children[-1]
    display(VBox([controls, output],  layout = Layout(flex_flow='columns wrap', width='max-content', size=10)))
    
#https://www.eia.gov/energyexplained/units-and-calculators/
btu_per_oil_bbl = 5698000
PJ_per_oil_bbl = btu_per_oil_bbl*1.05506e-12

## Overview

Hydrogen infrastructure in the OEO database includes pathways for hydrogen production, transport, conversion and utilization in the end-use sectors. These pathways are described in the sections that follow. 

### Hydrogen production
We consider four separate pathways to produce hydrogen.

#### Electrolysis 
In the electrolysis pathway ("ELEC_H2"), energy is consumed in the form of electricity to produce hydrogen (we do not consider water use). Process efficiencies, capital and fixed costs are based on data from the IEA Future of Hydrogen Report [(Annex: Production pathways table)](https://iea.blob.core.windows.net/assets/29b027e5-fefc-47df-aed0-456b1bb38844/IEA-The-Future-of-Hydrogen-Assumptions-Annex_CORR.pdf). 

#### Natural gas steam methane reforming  
In the steam methane reforming pathway ("NG_SMR"), natural gas is consumed to produce hydrogen. Since this is a mature, commercial process, associated techno-economic data are reported to be constant over the modeling time frame. Process efficiencies, capital and fixed costs are based on data from the IEA Future of Hydrogen Report [(Annex: Production pathways table)](https://iea.blob.core.windows.net/assets/29b027e5-fefc-47df-aed0-456b1bb38844/IEA-The-Future-of-Hydrogen-Assumptions-Annex_CORR.pdf) as well.

#### Natural gas steam methane reforming  with carbon capture and storage
In the steam methane reforming pathway with carbon capture and storage ("NG_SMR_CCS"), natural gas is consumed to produce hydrogen. In addition, emissions from natural gas combustion are captured and can be either sequestered or utilized elsewhere within the energy system. Process efficiencies, capital and fixed costs are based on data from the IEA Future of Hydrogen Report [(Annex: Production pathways table)](https://iea.blob.core.windows.net/assets/29b027e5-fefc-47df-aed0-456b1bb38844/IEA-The-Future-of-Hydrogen-Assumptions-Annex_CORR.pdf) as well. 

#### Bioenergy with carbon capture and storage 
In this pathway ("BECCS_H2_N"), biomass (herbaceous, woody and waste) is used in a gasification process to produce hydrogen, with electricity as an additional input. Around 80\% of the associated CO<sub>2</sub> produced is captured and either sequestered or utilized elsewhere in the energy system. Technoeconomic parameters for this process are drawn from the Princeton Net-Zero America report [(Annex L, Appendix A)](https://netzeroamerica.princeton.edu/img/NZA%20Annex%20L%20-%20Hydrogen%20and%20synthesized%20fuels.pdf).

#### Hydrogen production technoeconomic data
Efficiency and capital cost data for the hydrogen production processes can be viewed in the tables below. 

In [2]:
query = "SELECT vintage, tech, efficiency FROM Efficiency WHERE output_comm = 'hydrogen'"
df_efficiency = pd.read_sql_query(query, con)
df_efficiency = df_efficiency.drop_duplicates()
df_efficiency['efficiency'] = np.round(df_efficiency['efficiency'],2)*100
df_efficiency = df_efficiency.pivot_table(index=['tech'], columns=[ 'vintage'], \
                                         values='efficiency').reset_index()
df_efficiency.fillna('-', inplace=True)
header = ['technology', '2020', '2025', '2030', '2035', '2040','2045', '2050']
display(Markdown('**Table 1.** Hydrogen production pathways - efficiency (%)'))
display(HTML(tabulate.tabulate(df_efficiency.set_index('tech'), header,  floatfmt=".0f" , tablefmt='html')))

**Table 1.** Hydrogen production pathways - efficiency (%)

technology,2020,2025,2030,2035,2040,2045,2050
BECCS_H2_N,-,-,56,56,56,56,56
ELEC_H2,64.0,66.0,69,70,72,73,74
NG_SMR,76.0,76.0,76,76,76,76,76
NG_SMR_CCS,69.0,69.0,69,69,69,69,69


In [3]:
query = "SELECT vintage, tech, cost_invest FROM CostInvest WHERE tech IN ('" + \
"','".join(df_efficiency['tech'].unique()) + "')"
df_efficiency = pd.read_sql_query(query, con)
df_efficiency = df_efficiency.drop_duplicates()
df_efficiency['efficiency'] = np.round(df_efficiency['cost_invest'])
df_efficiency = df_efficiency.pivot_table(index=['tech'], columns=[ 'vintage'], \
                                         values='efficiency').reset_index()
df_efficiency.fillna('-', inplace=True)
header = ['technology', '2020', '2025', '2030', '2035', '2040','2045', '2050']
display(Markdown('**Table 2.** Hydrogen production pathways - capital costs (\$M/(PJ/year))'))
display(HTML(tabulate.tabulate(df_efficiency.set_index('tech'), header,  floatfmt=".0f" , tablefmt='html')))


**Table 2.** Hydrogen production pathways - capital costs (\$M/(PJ/year))

technology,2020,2025,2030,2035,2040,2045,2050
BECCS_H2_N,-,-,82,82,82,82,82
ELEC_H2,45.0,38.0,32,29,26,22,19
NG_SMR,29.0,29.0,29,29,29,29,29
NG_SMR_CCS,53.0,48.0,43,42,42,41,41


### Hydrogen transport

#### Hydrogen distribution
We consider hydrogen distribution via pipelines only, based on data from the [IEA Future of Hydrogen 2019 report](https://iea.blob.core.windows.net/assets/9e3a3493-b9a6-4b7d-b499-7ca48e357561/The_Future_of_Hydrogen.pdf). This report suggests that while trucks will remain the primary source of hydrogen distribution in the next decade, higher hydrogen demand will likely require distribution pipelines. Further, given sufficient demand, pipelines will likely be the cheapest option for distribution distances of less than 1500 km. For simplicity, therefore, we only consider hydrogen distribution via pipelines, with no efficiency losses, and at an average pressure of 10 bar. 
While costs per unit of hydrogen transported vary depending on pipeline size, data presented in the IEA report suggests that transporting hydrogen in a 100 tpd pipeline over 250 km would have similar costs as transporting it in a 500 tpd pipeline over 500 km, about 0.5\$/kg H<sub>2</sub>. We assume this variable cost applies to all intra-regional hydrogen distribution starting in 2020.

#### Hydrogen transmission 
We consider hydrogen transmission via pipelines between different OEO regions at an average pressure of 100 bar. Capital cost and lifetime data for hydrogen transmission are drawn from the IEA Future of Hydrogen Report [(Annex: Transmission table)](https://iea.blob.core.windows.net/assets/29b027e5-fefc-47df-aed0-456b1bb38844/IEA-The-Future-of-Hydrogen-Assumptions-Annex_CORR.pdf). They report a pipeline capital cost of \$1.21M/km with a throughput of 340 ktH<sub>2</sub>/year and a lifetime of 40 years. The capital cost data per km are merged with average distances between OEO regions to represent capital costs on a $M/(PJ/year) basis between regions. Distances between the OEO regions are drawn from the PowerGenome tool, which is used to create input datasets for the electric sector. For simplicity, we assume transmission between regions with no efficiency losses. Costs are reported in the table below.

In [4]:
query = "SELECT regions, tech, cost_invest FROM CostInvest WHERE tech ='H2_TRANS_N'"
df_cost = pd.read_sql_query(query, con)
df_cost = df_cost.drop_duplicates()


df_cost['cost_invest'] = np.round(df_cost['cost_invest'],1)
df_cost.loc[:,'region 1'] = df_cost.regions.str.split('-').apply(lambda x: x[0])
df_cost.loc[:,'region 2'] = df_cost.regions.str.split('-').apply(lambda x: x[1])
    
df_cost = df_cost.pivot_table(index=['region 1'], columns=[ 'region 2'], \
                                         values='cost_invest').reset_index()
df_cost.fillna('-', inplace=True)

header = df_cost.columns[1:]
display(Markdown('**Table 2.** Inter-regional hydrogen transmission - capital costs (\$M/(PJ/year))'))
display(HTML(tabulate.tabulate(df_cost.set_index('region 1'), header, floatfmt=".0f" , tablefmt='html')))


**Table 2.** Inter-regional hydrogen transmission - capital costs (\$M/(PJ/year))

,CA,CEN,MID_AT,NE,NW,N_CEN,SE,SW,TX
CA,-,58.2,101.6,117.4,29.7,70.1,94.2,23.4,57.9
CEN,58.2,-,44.2,63.0,48.4,23.2,37.2,34.9,23.6
MID_AT,101.6,44.2,-,22.6,85.5,35.8,23.5,78.8,58.5
NE,117.4,63.0,22.6,-,96.9,47.6,45.6,95.7,80.1
NW,29.7,48.4,85.5,96.9,-,50.1,85.1,27.6,60.3
N_CEN,70.1,23.2,35.8,47.6,50.1,-,42.5,49.3,46.6
SE,94.2,37.2,23.5,45.6,85.1,42.5,-,70.9,41.8
SW,23.4,34.9,78.8,95.7,27.6,49.3,70.9,-,36.6
TX,57.9,23.6,58.5,80.1,60.3,46.6,41.8,36.6,-


### Hydrogen storage
We currently include a simplistic representation of underground hydrogen storage. We assume no leakage or losses, based on data summarized in [Colbertaldo et al, 2019](https://www.sciencedirect.com/science/article/abs/pii/S0360319918336449) that suggest low leakage rates in existing underground facilities. We assume a storage duration of 2000 hours (nearly 3 months). There is considerable variability in capital cost estimates for hydrogen storage - estimates for above ground storage tanks are around \\$12/kWh while [Amos, 1999](https://www.nrel.gov/docs/fy99osti/25106.pdf) suggests that underground storage could cost up to \\$0.35/kWh. 
Using a conservative estimate of \\$1.05/kWh along with the assumed storage duration, we estimate capital costs of around 730 \\$M/GW. We ignore any associated operating costs. 
We assume a discount (hurdle) rate of 6\% to reflect the lack of experience with this technology, broadly. Further, we assume a lifetime of 30 years. We currently do not model resource limits on available underground storage across the different OEO regions, although this is planned for future work. 

For more detail on how this process was modeled for the OEO, see [Eshraghi 2020](https://repository.lib.ncsu.edu/bitstream/handle/1840.20/37269/etd.pdf?sequence=1&isAllowed=y), Section B.7.6. 

### Hydrogen compression

#### 10 bar to 100 bar
Hydrogen produced at 10 bar and transmitted via distribution pipelines is assumed to be compressed to around 100 bar before it can be utilized or transported via high pressure pipelines. This explicitly does not account for any costs or savings that may be incurred by delivering hydrogen at a pressure fit for different end-uses. Process efficiency (i.e., electricity input requirements) are derived based on an estimate of the electricity required to drive an adiabatic compressor. An overall efficiency of 94\% is thus derived and represented in the TechInputSplit table in the OEO database. 

Investment costs are based on data from [Niaz et al, 2015](https://www.sciencedirect.com/science/article/pii/S1364032115004694), who report a range of 1200 - 1800 \\$/kW<sub>e</sub>. We use an average of 1500 $/kW<sub>e</sub>, which is then scaled to hydrogen output of 90 \\$/kW H<sub>2</sub>. We further apply a factor of 1.15 to account for installation costs based on [ENEA, 2016](https://www.enea-consulting.com/static/3663dbb115f833de23e4c94c8fa399ec/enea-the-potential-of-power-to-gas.pdf). Fixed O\&M costs are assumed to be 4\% of capital costs, which falls within the range reported by [Ferreroa et al, 2015](https://www.sciencedirect.com/science/article/pii/S1876610216312164) (2\%) and [ENEA, 2016](https://www.enea-consulting.com/static/3663dbb115f833de23e4c94c8fa399ec/enea-the-potential-of-power-to-gas.pdf) (6\%). We assume a technology lifetime of 15 years, based on data from [ENEA, 2016](https://www.enea-consulting.com/static/3663dbb115f833de23e4c94c8fa399ec/enea-the-potential-of-power-to-gas.pdf)

For more detail on how this process was modeled for the OEO, see [Eshraghi 2020](https://repository.lib.ncsu.edu/bitstream/handle/1840.20/37269/etd.pdf?sequence=1&isAllowed=y), Section B.7.5. 

#### 100 bar to 700 bar
Fuel cell vehicles typically store hydrogen onboard in tanks operating at a pressure of around 700 bar. Therefore hydrogen at 100 bar needs to be compressed before it can be used in the light and heavy duty vehicle fleet. Electricity requirements are estimated using an approach that is similar to what is presented above, for compression to 100 bar, resulting in a process efficiency of around 95\%. Compression costs are assumed to be 20\% higher than costs for compressing hydrogen to 100 bar, while the process lifetime remains 15 years. 

For more detail on how this process was modeled for the OEO, see [Eshraghi 2020](https://repository.lib.ncsu.edu/bitstream/handle/1840.20/37269/etd.pdf?sequence=1&isAllowed=y), Section B.7.7. 

#### Hydrogen liquefaction
Hydrogen produced at 10 bar can be liquefied and used in the heavy-duty transport sector. Technoeconomic  data for hydrogen liquefaction are primarily drawn from the IEA Future of Hydrogen Report [(Annex: Transmission table)](https://iea.blob.core.windows.net/assets/29b027e5-fefc-47df-aed0-456b1bb38844/IEA-The-Future-of-Hydrogen-Assumptions-Annex_CORR.pdf). Based on the report, we assume 6.1 kWh of electricity is required per kg H<sub>2</sub> as process input, which is represented in the TechInputSplit table in the OEO database. The capital and fixed cost assumptions are based on a typical facility with a capacity of 260 ktons H<sub>2</sub>/year.

### Hydrogen utilization

#### Industrial boilers
For details on the parameterization of how hydrogen can be used to in conventional boilers in the industrial sector, please refer to the IndustrialSector notebook in this directory.

#### Industrial process heat
For details on the parameterization of how hydrogen can be used to provide industrial process heat, please refer to the IndustrialSector notebook in this directory.

#### Combined-cycle turbines
For details on the parametrization of hydrogen combined-cycle turbines in the power system, please refer to the ElectricSector notebook in this directory.

#### F-T fuels
We include a pathway that that generates liquid fuels from hydrogen and CO<sub>2</sub> via the Fischer-Tropsch (FT) process, which can be used as carbon-free liquid fuel in the transportation sector. Electricity is also required as an energy input to this process. Process parameterization is based on data from [Zang et al, 2021](https://www.sciencedirect.com/science/article/abs/pii/S2212982021000263). We assume a 52\% efficiency on an energy basis. Further, we assume a range of fuels can be produced via this process including diesel, jet fuel, LPG, residual fuel oil and marine gas oil and that the process is flexible enough to produce any of these commodities based on internal model demand; i.e., we do not assumed a fixed share of output fuels.
Capital and fixed costs are also based on [Zang et al, 2021](https://www.sciencedirect.com/science/article/abs/pii/S2212982021000263). The report a plant capacity of 16.5 TJ/year and associated capital cost of \\$378 million and annual fixed costs of \\$20 million. They suggest that the variable costs are dominated by feedstock costs, and therefore we ignore any additional process related variable costs. The techno-economic input data are summarized in the table below. The ratio of CO<sub>2</sub>, hydrogen and electricity inputs are specified in the TechInputSplit table. 

In [5]:
query = "SELECT efficiency*100 FROM Efficiency WHERE tech = 'T_FT' AND input_comm LIKE '%H2%'"
df_eff = pd.read_sql_query(query, con)
df_eff.drop_duplicates(inplace=True)

query = "SELECT cost_invest FROM CostInvest WHERE tech = 'T_FT'"
df_cap = pd.read_sql_query(query, con)
df_cap.drop_duplicates(inplace=True)
df_cap


query = "SELECT cost_fixed FROM CostFixed WHERE tech = 'T_FT'"
df_fom = pd.read_sql_query(query, con)
df_fom.drop_duplicates(inplace=True)
df_fom

query = "SELECT life FROM LifetimeTech WHERE tech = 'T_FT'"
df_life = pd.read_sql_query(query, con)
df_life.drop_duplicates(inplace=True)

data = [str(int(df_eff['efficiency*100'][0])), 
df_cap['cost_invest'][0],
df_fom['cost_fixed'][0], str(int(df_life['life'][0]))]

df_data = pd.DataFrame(data, columns = ['Parameter value'], index = ['Efficiency','Capital cost', 'Fixed O&M', 'Lifetime'])
df_data.loc[:,'Units'] = ['%', '$M/(PJ/year)',  '$M/(PJ/year)', 'years']
df_data

,Parameter value,Units
Efficiency,52,%
Capital cost,63.01,$M/(PJ/year)
Fixed O&M,3.35,$M/(PJ/year)
Lifetime,40,years


#### Methanol

We include a pathway that that generates methanol from hydrogen and CO<sub>2</sub>, which can be used as carbon-free liquid fuel in the transportation sector. The 2020 efficiency of 80.4\% is drawn from [Pérez-Fortes et al. 2016](https://www.sciencedirect.com/science/article/pii/S0306261915009071), and we model minor improvements until 2040 up to 82\% efficiency. Technology capital costs for 2015 are drawn from [Atsonios et al. 2016](https://www.sciencedirect.com/science/article/abs/pii/S0360319915302718). [ENEA, 2016](https://www.enea-consulting.com/static/3663dbb115f833de23e4c94c8fa399ec/enea-the-potential-of-power-to-gas.pdf) suggests that capital costs may decline by a third by 2030 and by around 53% by 2050. Both projections are used to develop capital costs within the database. Fixed O\&M costs are also drawn from [ENEA, 2016](https://www.enea-consulting.com/static/3663dbb115f833de23e4c94c8fa399ec/enea-the-potential-of-power-to-gas.pdf), which projects that they would likely be 7.5\% of the capital costs, similar to values reported in [Atsonios et al. 2016](https://www.sciencedirect.com/science/article/abs/pii/S0360319915302718). Variable costs are attributed primarily to CO<sub>2</sub> at an additional cost of 10\$/ton CO<sub>2</sub> which is converted to $M/PJ MEOH based on the stoichiometry of the reaction that converts CO<sub>2</sub> to methanol. This variable cost is a relic from a previous version of the database and we plan to update this data in future database iterations. We assume a technology lifetime of 20 years, based on data from [ENEA, 2016](https://www.enea-consulting.com/static/3663dbb115f833de23e4c94c8fa399ec/enea-the-potential-of-power-to-gas.pdf). The ratio of CO<sub>2</sub> and hydrogen inputs are specified in the TechInputSplit table. For more detail on how this process was modeled for the OEO, see [Eshraghi 2020](https://repository.lib.ncsu.edu/bitstream/handle/1840.20/37269/etd.pdf?sequence=1&isAllowed=y), Section B.7.13. 

We also include a simple representation of methanol storage at the production site. Since methanol is stored as a liquid around ambient temperatures, we assume that methanol storage costs are negligble compared to other system costs. Energy losses are primarily due to pumping, and are therefore assumed negligible as well. Further, a 20 year process lifetime is assumed. For more detail on how methanol storage was modeled for the OEO, see [Eshraghi 2020](https://repository.lib.ncsu.edu/bitstream/handle/1840.20/37269/etd.pdf?sequence=1&isAllowed=y), Section B.7.14. 

Methanol can be blended with gasoline and used as a lower-carbon transportation fuel, as E10 or E85. E10 is assumed to be blended as 15% methanol and 85% gasoline, while E85 can accept a minimum blending value of 80% methanol. Further details on transportation vehicles that use E10 and E85 are described in the TransportationSector notebook in this directory. For more detail on how this process was modeled for the OEO, see [Eshraghi 2020](https://repository.lib.ncsu.edu/bitstream/handle/1840.20/37269/etd.pdf?sequence=1&isAllowed=y), Section B.7.15. 

#### Synthetic natural gas

We include a pathway that that generates synthetic natural gas (SNG) from hydrogen and CO<sub>2</sub> via the Sabatier process, which can be used as carbon-free gaseous fuel in several of the end-use sectors, as a replacement for fossil-based natural gas. We assume a process efficiency of 80\% in 2020, based on data from  [ENEA, 2016](https://www.enea-consulting.com/static/3663dbb115f833de23e4c94c8fa399ec/enea-the-potential-of-power-to-gas.pdf), [Gotz et al, 2016](https://www.sciencedirect.com/science/article/pii/S0960148115301610) and [Ahern et al, 2015](https://www.sciencedirect.com/science/article/abs/pii/S096014811500066X). [Schiebahn et al, 2015](https://www.sciencedirect.com/science/article/abs/pii/S0360319915001913) suggest that the maximum  efficiency of the Sabatier process can reach 83\%, and so we assume that process efficiency improves slightly to 81\% by 2030. We also assume that this synthetic gas is produced at 20 bar.

Technology capital costs for 2020 are drawn from [Gotz et al, 2016](https://www.sciencedirect.com/science/article/pii/S0960148115301610) who report a range of 360 to 720 \\$/kW: a median value of 540 \$/kW is assumed. [ENEA, 2016](https://www.enea-consulting.com/static/3663dbb115f833de23e4c94c8fa399ec/enea-the-potential-of-power-to-gas.pdf) suggests that capital costs may decline to a third by 2050. More conservatively, we assume costs may fall to half by 2040. Fixed O\&M costs are also drawn from [ENEA, 2016](https://www.enea-consulting.com/static/3663dbb115f833de23e4c94c8fa399ec/enea-the-potential-of-power-to-gas.pdf), which projects that they would likely be 7.5\% of the capital costs. Variable costs are attributed primarily to CO<sub>2</sub> at an additional cost of 10\$/ton CO<sub>2</sub> which is converted to $M/PJ SNG based on the stoichiometry of the reaction that converts CO<sub>2</sub> to methane. This variable cost is a relic from a previous version of the database and we plan to update this data in future database iterations. We assume a technology lifetime of 20 years, based on data from [ENEA, 2016](https://www.enea-consulting.com/static/3663dbb115f833de23e4c94c8fa399ec/enea-the-potential-of-power-to-gas.pdf). The ratio of CO<sub>2</sub> and hydrogen inputs are specified in the TechInputSplit table. For more detail on how this process was modeled for the OEO, see [Eshraghi 2020](https://repository.lib.ncsu.edu/bitstream/handle/1840.20/37269/etd.pdf?sequence=1&isAllowed=y), Section B.7.10. 

The produced gas at 20 bar will typically be compressed before it can be used within the existing natural gas transmission pipeline infrastructure which can be pressurized at anywhere between 16 and 100 bar. As a conservative estimate, we assume the produced gas will be pressurized to 100 bar. For more detail on how this process was modeled for the OEO, see [Eshraghi 2020](https://repository.lib.ncsu.edu/bitstream/handle/1840.20/37269/etd.pdf?sequence=1&isAllowed=y), Section B.7.11. 

#### Fuel cell vehicles and liquid hydrogen
For details on the parameterization of fuel cell vehicles, liquid hydrogen use and hydrogen refueling infrastructure for transportation, please refer to the TransportationSector notebook in this directory.

## Technology/commodity description lookup tool <a class="anchor" id="description_look_up"></a>
Use the tool below to search for any key words that may describe a technology or commodity of interest (e.g. hydrogen). The tool provides a list of all the technologies or commodities in the database that may be relevant to the query.


In [6]:
w = widgets.Text(value='hydrogen compression')
display(w)
def f(w):
    if len(w)>0:
        df1 = pd.read_sql("SELECT * FROM commodities WHERE comm_desc LIKE '%" + w + "%'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech_desc LIKE '%" + w + "%'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'


        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])
        
        if len(df)>0:
            display(HTML(tabulate.tabulate(df.set_index('comm_tech'),['technology/commodity','type','description'],stralign='left',tablefmt='html')))
        else:
            print('')
    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='hydrogen compression')

Output()

## Network diagram lookup tool  <a class="anchor" id="network_look_up"></a>
Use the [description lookup tool](#description_look_up) above to identify specific commodity or technology names. Type the name in the box below to generate a corresponding network diagram for that commodity or technology. The slider can be used to view different upstream levels of the network diagram.

In [7]:
w = widgets.Text(value='hydrogen')
display(w)
def f(w):
    if len(w)>0:
        
        df1 = pd.read_sql("SELECT comm_name, comm_desc FROM commodities WHERE comm_name='" + w + "'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'

        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])

    
        if len(df)>0:
            def show_desc(level):
                display(Markdown(df['desc'][0]))
                final_dem = df['comm_tech'][0]
                df_graph = return_flowd_table(final_dem,level)
                args = create_args_flowd(df_graph)
                colors, quick_run_dot_fmt = return_format_colors()
                args.update(colors)
                #o_str = 'rankdir = "LR" ;'
                #r_str = 'rankdir = "LR" ; \n\t size="8,8";'
                #quick_run_dot_fmt = quick_run_dot_fmt.replace(o_str, r_str)
                dot_graph = quick_run_dot_fmt % args
                display(graphviz.Source(dot_graph))
            w2 = widgets.IntSlider(value=1,min=0,max=10,step=1,description='Level:',disabled=False,continuous_update=True,orientation='horizontal',readout=True,readout_format='d')
            w = widgets.interactive(show_desc, level=w2)
            controls_rows(w)
        else:
            print('')

    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='hydrogen')

Output()